In [3]:
# # uncomment and run below:
# %pip install -qU langchain
# %pip install -qU langchain-openai
# %pip install tiktoken
# %pip install faiss-cpu
# %pip install beautifulsoup4
# %pip install google-search-results
# %pip install pydantic

In [1]:
import os
import getpass

# Set OPENAI API Key


import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"var: ")

_set_env("OPENAI_API_KEY")

OpenAI API quickstart: https://platform.openai.com/docs/quickstart

# Introduction to LangChain 

Working with LLMs involves in one way or another working with a specific type of abstraction: "Prompts".

However, in the practical context of day-to-day tasks we expect LLMs to perform, these prompts won't be some static and dead type of abstraction. Instead we'll work with dynamic prompts re-usable prompts.

# Lanchain

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) is a framework that allows you to connect LLMs together by allowing you to work with modular components like prompt templates and chains giving you immense flexibility in creating tailored solutions powered by the capabilities of large language models.


Its main features are:
- **Components**: abstractions for working with LMs
- **Off-the-shelf chains**: assembly of components for accomplishing certain higher-level tasks

LangChain facilitates the creation of complex pipelines that leverage the connection of components like chains, prompt templates, output parsers and others to compose intricate pipelines that give you everything you need to solve a wide variety of tasks.

At the core of LangChain, we have the following elements:

- Models
- Prompts
- Output parsers

**Models**

Models are nothing more than abstractions over the LLM APIs like the OpenAI API.​

In [7]:
from langchain.chat_models import init_chat_model

In [9]:
MODEL = "gpt-4o-mini"
chat_model = init_chat_model(model=MODEL, temperature=0)

Temperature is a measure of how random a response is, so a number close to 0 means a "precise" answer, and a number closer to 1 or 2 means a more "creative"
output, that is the superficial explanation.

In [10]:
prompt = "I am teaching a live-training about LLMs!"
output = chat_model.invoke(prompt)
output

AIMessage(content="That sounds exciting! Large Language Models (LLMs) are a fascinating topic with a lot of depth. What specific aspects are you planning to cover in your training? Here are some ideas you might consider including:\n\n1. **Introduction to LLMs**: Explain what LLMs are, how they work, and their significance in the field of AI.\n\n2. **Architecture**: Discuss the architecture of popular LLMs like GPT, BERT, and others. You could touch on concepts like transformers, attention mechanisms, and training processes.\n\n3. **Applications**: Highlight various applications of LLMs, such as chatbots, content generation, translation, summarization, and more.\n\n4. **Ethical Considerations**: Address the ethical implications of using LLMs, including bias, misinformation, and the environmental impact of training large models.\n\n5. **Hands-On Demonstration**: If possible, include a live coding session or demonstration using an LLM API (like OpenAI's GPT) to show how to interact with t

In [11]:
type(output)

langchain_core.messages.ai.AIMessage

In [12]:
print(output.content)

That sounds exciting! Large Language Models (LLMs) are a fascinating topic with a lot of depth. What specific aspects are you planning to cover in your training? Here are some ideas you might consider including:

1. **Introduction to LLMs**: Explain what LLMs are, how they work, and their significance in the field of AI.

2. **Architecture**: Discuss the architecture of popular LLMs like GPT, BERT, and others. You could touch on concepts like transformers, attention mechanisms, and training processes.

3. **Applications**: Highlight various applications of LLMs, such as chatbots, content generation, translation, summarization, and more.

4. **Ethical Considerations**: Address the ethical implications of using LLMs, including bias, misinformation, and the environmental impact of training large models.

5. **Hands-On Demonstration**: If possible, include a live coding session or demonstration using an LLM API (like OpenAI's GPT) to show how to interact with the model.

6. **Best Practice

You can predict outputs from both LLMs and ChatModels:

Basic components are:

- Models
- Prompt templates
- Output parsers

In [13]:
from langchain_core.prompts import ChatPromptTemplate

In [14]:
template = "Show me 5 examples of this concept: {concept}"

prompt = ChatPromptTemplate.from_template(template)

prompt.format(concept="animal")

'Human: Show me 5 examples of this concept: animal'

In [15]:
chain = prompt | chat_model

In [16]:
type(chain)

langchain_core.runnables.base.RunnableSequence

In [17]:
output = chain.invoke({"concept": "wooden object"})

In [18]:
output.content

'Sure! Here are five examples of wooden objects:\n\n1. **Dining Table**: A sturdy table made from hardwood, often used for family meals and gatherings.\n\n2. **Wooden Chair**: A classic piece of furniture, typically crafted from oak, maple, or pine, designed for seating.\n\n3. **Wooden Toy**: A handcrafted toy, such as a wooden train or building blocks, often made from safe, non-toxic materials for children.\n\n4. **Wooden Cutting Board**: A kitchen essential made from hardwood, used for chopping and preparing food.\n\n5. **Wooden Jewelry Box**: A decorative box made from various types of wood, often used to store jewelry or small keepsakes.'

In [19]:
from IPython.display import Markdown


Markdown(output.content)

Sure! Here are five examples of wooden objects:

1. **Dining Table**: A sturdy table made from hardwood, often used for family meals and gatherings.

2. **Wooden Chair**: A classic piece of furniture, typically crafted from oak, maple, or pine, designed for seating.

3. **Wooden Toy**: A handcrafted toy, such as a wooden train or building blocks, often made from safe, non-toxic materials for children.

4. **Wooden Cutting Board**: A kitchen essential made from hardwood, used for chopping and preparing food.

5. **Wooden Jewelry Box**: A decorative box made from various types of wood, often used to store jewelry or small keepsakes.

You can also use the predict method over a string input:

In [20]:
text = "What would be a good name for a dog that loves to nap??"
chat_model.invoke(text)

AIMessage(content="Here are some cute name ideas for a dog that loves to nap:\n\n1. Snoozer\n2. Naptime\n3. Dreamer\n4. Dozer\n5. Snuggles\n6. Siesta\n7. Zzz\n8. Pillow\n9. Drowse\n10. Napster\n\nChoose one that fits your dog's personality!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 21, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CBiZRkgheVTUeikZ1HhUCOHTJczVl', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--2a0679ce-c072-4d40-9ce5-fb28eb849c80-0', usage_metadata={'input_tokens': 21, 'output_tokens': 74, 'total_tokens': 95, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 

**Prompts**

The same works for prompts. Now, prompts are pieces of text we feed to LLMs, and LangChain allows you to work with prompt templates.

Prompt Templates are useful abstractions for reusing prompts and they are used to provide context for the specific task that the language model needs to complete. 

A simple example is a `PromptTemplate` that formats a string into a prompt:

In [21]:
from langchain_core.prompts  import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("What is a good dog name for a dog that loves to {activity}?")
prompt.format(activity="sleeping")
# Output: "What is a good dog name for a dog that loves to nap?"

'Human: What is a good dog name for a dog that loves to sleeping?'

In [22]:
chain = prompt | chat_model

chain.invoke({'activity': 'sleeping'})

AIMessage(content="Here are some cute and fitting names for a dog that loves to sleep:\n\n1. **Snoozer**\n2. **Napster**\n3. **Dozer**\n4. **Slumber**\n5. **Dreamer**\n6. **Pillow**\n7. **Cuddles**\n8. **Resty**\n9. **Zzz**\n10. **Naptime**\n\nChoose one that resonates with your dog's personality!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 21, 'total_tokens': 109, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CBiZX7BesiuthiNW58pZDcC6D54K5', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--7fd864a3-2174-4bdc-b3b4-6a587ec3bfac-0', usage_metadata={'input_tokens': 21, 'output_tokens': 88, 'total_tokens': 109, 'input_token_details': {'aud

**Output Parsers**

OutputParsers convert the raw output from an LLM into a format that can be used downstream. Here is an example of an OutputParser that converts a comma-separated list into a list:

In [23]:
chain.invoke({"activity": "Landscapes"})

AIMessage(content='A great name for a dog that loves landscapes could be "Scenic." Other options might include "Vista," "Meadow," "Willow," "Canyon," or "Pine." These names evoke the beauty of nature and the outdoors, reflecting your dog\'s love for landscapes!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 22, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CBiZcLVCncsPyx07TYVdLJO9UOHXs', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--94c0f91d-e007-4068-84fe-d70c32446fd7-0', usage_metadata={'input_tokens': 22, 'output_tokens': 57, 'total_tokens': 79, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token

In [24]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [25]:
MODEL = "gpt-4o-mini"
llm = init_chat_model(model=MODEL, temperature=0.0)
prompt = ChatPromptTemplate.from_template("""
Write 5 concepts that are fundamental to learn about {topic}.
                                          """)
chain = prompt | llm | output_parser
chain.invoke({"topic": "Artificial Neural Networks"})

"Certainly! Here are five fundamental concepts that are essential to understand when learning about Artificial Neural Networks (ANNs):\n\n1. **Neurons and Activation Functions**:\n   - At the core of ANNs are artificial neurons, which are inspired by biological neurons. Each neuron receives inputs, processes them, and produces an output. The output is typically passed through an activation function, which introduces non-linearity into the model. Common activation functions include Sigmoid, ReLU (Rectified Linear Unit), and Tanh. Understanding how these functions work and their impact on the network's performance is crucial.\n\n2. **Network Architecture**:\n   - The architecture of an ANN refers to its structure, including the number of layers (input, hidden, and output layers) and the number of neurons in each layer. Different architectures can be used for different tasks (e.g., feedforward networks, convolutional neural networks, recurrent neural networks). Learning how to design and 

This chain will take input variables, pass those to a prompt template to create a prompt, pass the prompt to an LLM, and then pass the output through an output parser.

Ok, so these are the basics of langchain. But how can we leverage these abstraction capabilities inside our LLM app application?

Now, to put everything together LangChain allows you to build something called "chains", which are components that connect prompts, llms and output parsers into a building block that allows you to create more interesting and complex functionality.

Let's look at the example below:

So, what the chain is doing is connecting these basic components (the LLM and the prompt template) into
a block that can be run separately. The chain allows you to turn workflows using LLLMs into this modular process of composing components.

Now, the newer versions of LangChain have a new representation language to create these chains (and more) known as LCEL or LangChain expression language, which is a declarative way to easily compose chains together. The same example as above expressed in this LCEL format would be:

In [26]:
chain = prompt | llm

chain.invoke({"topic": "sleep"})

AIMessage(content="Certainly! Here are five fundamental concepts to understand about sleep:\n\n1. **Sleep Stages and Cycles**: Sleep is divided into several stages, primarily categorized into Non-Rapid Eye Movement (NREM) and Rapid Eye Movement (REM) sleep. A typical sleep cycle lasts about 90 minutes and includes multiple stages, each serving different functions for physical and mental health. Understanding these stages helps in recognizing the importance of both deep sleep (NREM) for physical restoration and REM sleep for cognitive functions like memory consolidation.\n\n2. **Circadian Rhythms**: The body's internal clock, or circadian rhythm, regulates the sleep-wake cycle over a 24-hour period. This biological process is influenced by external cues like light and temperature. Disruptions to circadian rhythms, such as shift work or irregular sleep patterns, can lead to sleep disorders and negatively impact overall health.\n\n3. **Sleep Hygiene**: Good sleep hygiene refers to practic

# Address some great questions!

## RT: Will you also cover about Tokens? how are Tokens getting utilized when we are creating multiple chains?

In [27]:
prompt = "Explain what are tokens in large language models"

MODEL = "gpt-4o-mini"
llm = init_chat_model(model=MODEL, temperature=0.0)
output = llm.invoke(prompt)


In [28]:
output.usage_metadata

{'input_tokens': 15,
 'output_tokens': 517,
 'total_tokens': 532,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

Relevant for calculating the cost of using this model for this particular task!

Cost of an LLM is calculated as a function of $/number of tokens.

Ollama download:
- [Donwload ollama](https://ollama.com/)

In [ ]:
# % pip install langchain-ollama

In [29]:
from langchain_ollama import ChatOllama

In [34]:
# ollama pull llama3 in the terminal
llm = ChatOllama(model="llama3.2")

In [35]:
type(llm)

langchain_ollama.chat_models.ChatOllama

In [36]:
llm.invoke("Say hello to my students on a course about LLMs!")

AIMessage(content="Hello students!\n\nI'm excited to be your guide on this journey into the world of Large Language Models (LLMs)! As we explore the capabilities and applications of these powerful artificial intelligence systems, I hope you'll discover new insights, develop practical skills, and maybe even uncover some fascinating secrets behind LLMs.\n\nThroughout our course, we'll delve into the fundamentals of language models, discuss the latest advancements in the field, and examine real-world use cases. We'll also explore challenges and limitations, as well as ways to apply LLMs to solve complex problems in natural language processing (NLP), machine learning, and beyond.\n\nWhether you're a beginner or looking to deepen your understanding of LLMs, I'm here to help you navigate the exciting landscape of this rapidly evolving field. So, let's get started on this adventure together!\n\nBefore we begin, I'd love to hear from you: What are your expectations from this course? Are there 

In [37]:
prompt = ChatPromptTemplate.from_template("""
Write 5 concepts that are fundamental to learn about {topic}.
                                          """)
chain = prompt | llm | output_parser

chain.invoke({"topic": "neuroscience of sleep"})

"Here are five fundamental concepts to learn about the neuroscience of sleep:\n\n1. **Sleep Cycles**: Sleep is not a single event, but rather a series of cycles consisting of different stages of sleep and wakefulness. A typical sleep cycle lasts around 90-120 minutes and consists of three stages: N1 (non-rapid eye movement), N2, and REM (rapid eye movement). Each cycle becomes progressively deeper, with the most restorative stage being REM.\n\n2. **Sleep-Wake Homeostasis**: The body's need for sleep is regulated by a homeostatic mechanism that builds up adenosine, a chemical that promotes sleepiness. As adenosine levels increase, the desire for sleep also increases. This mechanism helps to regulate the length and quality of sleep, with more sleep needed as adenosine levels build up.\n\n3. **Brain Waves and Brain States**: During different stages of sleep, brain waves change in frequency and amplitude. The four main types of brain waves are:\n\t* Alpha waves (8-12 Hz): seen during relax

Notice that now the output is an `AIMessage()` object, which represents LangChain's way to abstract the output from an LLM model like ChatGPT or others.

These building blocks and abstractions that LangChain provides are what makes this library so unique, because it gives you the tools you didn't know you need it to build awesome stuff powered by LLMs.

# Our First LangChain App

See `./1.1-langchain-app.py`

# LangChain Exercise

Let's create a simple chain for summarization of content. 

Your chain should:

- A prompt template with one or more variables
- A model like ChatGPT or other (you can use local models if you'd like, I recommend `ChatOllama` for that!)
- Optional: use output parsing or just fetch the string output at the end!

## Example Answer

Let's make use of the `ChatPromptTemplate` to abstract away the following pieces of the prompt: 
- `content` - the text content to be summarized  
- `summary_format` - the format in which we want the summary to be presented (like bullet points and so on).

In [29]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Summarize this: {content}. The output should be in the following format: {summary_format}.")

# We can look at a simple example to illustrate what that prompt is doing
prompt.format(content="This is a test.", summary_format="One word summary")

'Human: Summarize this: This is a test.. The output should be in the following format: One word summary.'

Ok, now that we have our prompt template done, let's load the llm and create a nice chain to put everything together. 

In [30]:
from langchain_openai import ChatOpenAI

llm_chat =  ChatOpenAI()
chain = prompt | llm_chat # This is the Pipe symbol! from LCEL that connect model to prompt!

Now, that we have our chain we can run some tests. The cool thing about working with LLMs is that you can use them to create examples for simple tests like this (avoiding the annoynace of searching online for some piece of text, copying and pasting etc...). So, let's generate a few examples of tests below:

In [31]:
num_examples = 3
examples = []
for i in range(num_examples):
    examples.append(llm_chat.invoke("Create a piece of text with 2 paragraphs about a random topic regarding human-machine interaction."))

examples

[AIMessage(content='One fascinating aspect of human-machine interaction is the concept of artificial intelligence and its ability to learn and adapt to human behavior. As AI technologies become more advanced, machines are able to anticipate human needs and preferences, making interactions more seamless and intuitive. This can be seen in the rise of virtual assistants like Siri and Alexa, which use machine learning algorithms to understand and respond to natural language commands. As these technologies continue to evolve, the potential for more personalized and efficient interactions between humans and machines will only grow.\n\nOn the other hand, there are concerns about the implications of increasing reliance on machines for tasks traditionally performed by humans. Some worry about the loss of human autonomy and decision-making abilities as machines become more integrated into our daily lives. Additionally, there are ethical considerations surrounding the use of AI in areas like heal

Nice! Now that we have our examples, let's run our chain on them and check out the results.

In [32]:
summary_format = "bullet points"

outputs = []
for ex in examples:
    outputs.append(chain.invoke({"content": ex, "summary_format": summary_format}))

# Let's display one example output
outputs[0]

AIMessage(content='- Artificial intelligence in human-machine interaction allows machines to learn and adapt to human behavior\n- AI technologies can anticipate human needs and preferences, making interactions more seamless and intuitive\n- Virtual assistants like Siri and Alexa use machine learning algorithms to understand and respond to natural language commands\n- Concerns exist about the implications of increasing reliance on machines, including loss of human autonomy and ethical considerations\n- It is important to consider the balance between the benefits and risks of relying on AI technologies in society', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 97, 'prompt_tokens': 457, 'total_tokens': 554, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125

Great! So it seems our chain worked and we generated some summaries! Let's visualize all the summaries generated in a neat way.

In [33]:
from IPython.display import Markdown

for i in range(num_examples):
    display(Markdown(f"Output {i} \n {outputs[i].content}"))
# Markdown(f"**Input**: {examples[0]}\n\n**Output**: {outputs[0]}")

Output 0 
 - Artificial intelligence in human-machine interaction allows machines to learn and adapt to human behavior
- AI technologies can anticipate human needs and preferences, making interactions more seamless and intuitive
- Virtual assistants like Siri and Alexa use machine learning algorithms to understand and respond to natural language commands
- Concerns exist about the implications of increasing reliance on machines, including loss of human autonomy and ethical considerations
- It is important to consider the balance between the benefits and risks of relying on AI technologies in society

Output 1 
 - Human-machine interaction is crucial in today's society due to increasing automation and technological advances
- Designing intuitive and user-friendly interfaces is a key challenge in this field
- Understanding human behavior, cognitive processes, and preferences is essential for creating effective interfaces
- Advancements in artificial intelligence and machine learning have enabled personalized interfaces that adapt to user behavior
- The development of personalized interfaces has made interactions more seamless and efficient
- The future of human-machine interaction will continue to evolve and shape how we interact with and rely on machines.

Output 2 
 - The relationship between humans and machines is becoming increasingly intertwined as technology advances.
- Artificial intelligence is a key area where this integration is evident, with machines designed to mimic human thought processes and behavior.
- Virtual assistants like Siri and Alexa are examples of AI that can understand and respond to human commands in a human-like manner.
- Ethical questions arise regarding human-machine interaction, such as prioritizing human safety over machine existence and machines making decisions independently.
- Society will need to address these complex issues as technology evolves to ensure beneficial human-machine interaction.

Great! Our summaries worked, and we were able to apply a given summary format to all of them.

LangChain is an extremely powerful library to work with abstractions like these and throughout this course we hope to give you a gliimpse of the cool stuff you can build with it.

# Introduction to LangChain Expression Language ([LCEL](https://python.langchain.com/docs/get_started/introduction))

LCEL is a declarative way to compose chains of components. 

What does that mean? Means its an easy way to put useful building blocks together.


Here's quick summary of the LangChain Expression Language (LCEL) page:

- LCEL Basics: Simplifies building complex chains from basic components using a unified interface and composition primitives.

- Unified Interface: Every LCEL object implements the Runnable interface, supporting common invocation methods like invoke, batch, stream, ainvoke, and more.

- Composition Primitives: LCEL provides tools for composing chains, parallelizing components, adding fallbacks, and dynamically configuring internal chain elements.

- Model Flexibility: LCEL allows for easy switching between different models and providers (like OpenAI or Anthropic), and runtime configurability of chat models or LLMs.

- Advanced Features: LCEL features things like logging intermediate results with LangSmith integration and adding fallback logic for enhanced reliability.

Ok, cool but what is a component?

A component is something that implements the `Runnable` protocol.


Ok....and what is that?

It's an object with some nice desirable features like:

- input and output schemas (describe what that object takes as input and the structure of its output)

Some nice methods are:

- `invoke` [ainvoke]
- `stream` [astream]
- `batch` [abatch]


Below is a list of common i/o types for each component:

![](./assets-resources/components_input_type_output_type.png)

[source for the image](https://python.langchain.com/docs/expression_language/interface)

In [34]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

output = model.invoke("hi")

In [35]:
type(output)

langchain_core.messages.ai.AIMessage

In [78]:
model.input_schema.model_json_schema()

{'$defs': {'AIMessage': {'additionalProperties': True,
   'description': 'Message from an AI.\n\nAIMessage is returned from a chat model as a response to a prompt.\n\nThis message represents the output of the model and consists of both\nthe raw output as returned by the model together standardized fields\n(e.g., tool calls, usage metadata) added by the LangChain framework.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'const': 'ai',
     'default': 'ai',
     'enum': ['ai'],
     'title': 'Type',
     'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Name'},
    'id': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'd

In [77]:
model.output_schema.model_json_schema()

{'$defs': {'AIMessage': {'additionalProperties': True,
   'description': 'Message from an AI.\n\nAIMessage is returned from a chat model as a response to a prompt.\n\nThis message represents the output of the model and consists of both\nthe raw output as returned by the model together standardized fields\n(e.g., tool calls, usage metadata) added by the LangChain framework.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'const': 'ai',
     'default': 'ai',
     'enum': ['ai'],
     'title': 'Type',
     'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Name'},
    'id': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'd

Let's look at a simple example.

In [38]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm_chat = ChatOpenAI()
prompt = ChatPromptTemplate.from_template(("Translate this {word} into {language}"))
output_parser = StrOutputParser()

chain = prompt | llm_chat | output_parser

chain.invoke({"word": "responsibility", "language": "Italian"})

'Questo compito'

Ok nice! So we put everything together using this [pipe](https://en.wikipedia.org/wiki/Pipeline_(Unix)) `|` symbol (or [unix pipe operator](https://en.wikipedia.org/wiki/Pipeline_(Unix)) if you want to get fancy) That's the power of the LCEL language, putting different components together through a simple interface.

[source](https://python.langchain.com/docs/how_to/sequence/)

Here's a bullet point summary of the key features and benefits of LangChain Expression Language (LCEL):

Declarative Composing: LCEL allows for easy composition of chains, ranging from simple "prompt + LLM" chains to complex ones with hundreds of steps.

Streaming Support: LCEL offers optimal time-to-first-token, enabling streaming of tokens from an LLM to a streaming output parser for quick, incremental output.

Async Support: Chains built with LCEL can be used both synchronously (e.g., in Jupyter notebooks for prototyping) and asynchronously (e.g., in a LangServe server), maintaining consistent code for prototypes and production.

Optimized Parallel Execution: LCEL automatically executes parallel steps in a chain (like fetching documents from multiple retrievers) in both sync and async interfaces, reducing latency.

Retries and Fallbacks: Users can configure retries and fallbacks for any part of the LCEL chain, enhancing reliability at scale. Streaming support for these features is in development.

Access to Intermediate Results: LCEL allows access to intermediate step results, useful for user updates or debugging. This feature includes streaming intermediate results and is available on all LangServe servers.

Input and Output Schemas: LCEL chains come with Pydantic and JSONSchema schemas, inferred from the chain's structure, which aid in validating inputs and outputs. This is a core part of LangServe.

Seamless LangSmith Tracing Integration: As chains become more complex, LCEL provides automatic logging of all steps to LangSmith for enhanced observability and debuggability.

Seamless LangServe Deployment Integration: LCEL chains can be easily deployed using LangServe, facilitating smoother deployment processes.

These features highlight LCEL's versatility and efficiency in both development and production environments, making it a powerful tool for creating and managing complex language chains.